# Part II — Core Django  
## 11. Forms (HTML Forms, Validation, UX) — From Basics to Production Patterns

Django Forms are one of the biggest “superpowers” in Django because they give you a
structured, testable way to:

- accept user input (POST data, files)
- validate and normalize it (turn strings into Python types)
- show errors back to the user safely
- save to the database cleanly (ModelForms)
- prevent a whole class of security and correctness bugs

This chapter upgrades your project from “read-only pages” to **real create/edit
workflows** with proper UX and security.

---

## 11.0 Learning Outcomes

By the end of this chapter you will be able to:

1. Explain how HTML forms work with HTTP (GET vs POST, encoding types).
2. Explain Django’s CSRF protection and implement it correctly in templates.
3. Build and use:
   - `forms.Form` (pure validation / non-model input)
   - `forms.ModelForm` (create/edit database objects)
4. Understand the form lifecycle:
   - unbound form (GET)
   - bound form (POST)
   - `is_valid()`, `cleaned_data`, `errors`
5. Implement field-level and form-level validation:
   - `clean_<field>()`
   - `clean()`
6. Render forms in templates with:
   - `{{ form.as_p }}` (quick)
   - manual rendering (industry standard for real UI)
7. Use the Post/Redirect/Get pattern (PRG) to prevent double submissions.
8. Use Django messages framework to give feedback (“Saved!”).
9. Handle file uploads correctly (what changes in form + view + settings).
10. Write tests for form views: success path, validation errors, permission checks.

---

## 11.1 HTML Forms: What’s Actually Sent Over the Network

### 11.1.1 A form is just an HTTP request generator
When you submit a form, the browser makes an HTTP request to the form’s `action`
URL using the specified `method`.

Example HTML:

```django
<form method="post" action="/articles/new/">
  <input name="title" />
  <button type="submit">Save</button>
</form>
```

Browser sends a POST request:

- URL: `/articles/new/`
- Method: POST
- Body: key/value fields (often `application/x-www-form-urlencoded`)

### 11.1.2 `method="get"` vs `method="post"`
- GET forms:
  - put data in the query string
  - good for search/filtering (`?q=...&tag=...`)
  - should not change server state
- POST forms:
  - put data in the request body
  - used for create/update/delete actions
  - should be protected with CSRF

### 11.1.3 Encoding types (why file uploads need special handling)
Most forms use:

- `application/x-www-form-urlencoded` (default)
- `multipart/form-data` (required for file uploads)

If you include `<input type="file">`, you must use:

```html
<form method="post" enctype="multipart/form-data">
```

Otherwise the file will not be transmitted correctly.

---

## 11.2 CSRF: Why It Exists and How Django Enforces It

### 11.2.1 The problem: cookies are sent automatically
If your authentication is cookie-based (typical Django session login), a malicious
site can cause a user’s browser to send a POST request to your site while logged in.

### 11.2.2 Django’s defense: CSRF token
Django expects a secret token on unsafe requests (POST/PUT/PATCH/DELETE), usually:

- as a hidden input in the form body, generated by `{% csrf_token %}`

If missing or invalid, Django rejects the request (typically 403).

### 11.2.3 Practical rule
Every HTML form that performs POST **must** include:

```django
{% csrf_token %}
```

---

## 11.3 Django Forms: Why They’re Better Than Reading `request.POST` Directly

You *can* do:

```python
title = request.POST.get("title", "").strip()
if not title:
    ...
```

But this approach becomes messy quickly:
- repeated validation logic
- inconsistent error messages
- hard to test
- difficult to render errors next to fields consistently
- easy to forget edge cases (types, lengths, required fields)

Django Forms give you:
- declarative fields
- consistent validation
- normalized types
- structured errors you can display in templates

---

## 11.4 `forms.Form` (Pure Input Validation; Not Tied to a Model)

Use `forms.Form` when:
- input does not map 1:1 to a model
- you’re building a search/filter form
- you’re building a “contact us” form
- you’re validating webhook payload inputs (sometimes)
- you want a clear boundary between input validation and persistence

### 11.4.1 Example: a contact form (no model)

Create `pages/forms.py`:

```python
from django import forms


class ContactForm(forms.Form):
    name = forms.CharField(max_length=100)
    email = forms.EmailField()
    message = forms.CharField(min_length=10, widget=forms.Textarea)

    def clean_name(self) -> str:
        name = self.cleaned_data["name"].strip()
        if len(name.split()) < 1:
            raise forms.ValidationError("Please enter your name.")
        return name

    def clean(self):
        cleaned = super().clean()
        message = cleaned.get("message", "")
        if "http://" in message or "https://" in message:
            raise forms.ValidationError(
                "Please do not include links in the message."
            )
        return cleaned
```

#### Explanation (key concepts)
- Each field converts and validates input:
  - `EmailField` ensures a valid email shape.
  - `CharField(min_length=...)` ensures minimum length.
- `clean_<field>()` runs after built-in field cleaning and lets you refine/validate.
- `clean()` is form-level validation, used for cross-field checks or policy rules.
- Errors raised via `forms.ValidationError` become displayable form errors.

---

## 11.5 `forms.ModelForm` (The Professional Way to Create/Edit Models)

Use `ModelForm` when:
- you are creating/editing a Django model instance
- you want field definitions generated from the model
- you want consistent validation and clean saving

We will build create/edit flows for your `Article` model.

---

## 11.6 Build an `ArticleForm` (ModelForm) With Real Validation

Your `Article` model has:
- `status` with choices
- `published_at` optional in schema, but constrained:
  - published requires published_at (DB check constraint)

If you don’t validate this in the form, users can submit “Published” without a
timestamp and hit an `IntegrityError` at save time (bad UX).

So we enforce it in the form.

### 11.6.1 Create `articles/forms.py`
Create `articles/forms.py`:

```python
from django import forms
from django.utils import timezone
from django.utils.text import slugify

from .models import Article, Tag


class ArticleForm(forms.ModelForm):
    tags = forms.ModelMultipleChoiceField(
        queryset=Tag.objects.all().order_by("name"),
        required=False,
        widget=forms.CheckboxSelectMultiple,
        help_text="Select tags for this article (optional).",
    )

    class Meta:
        model = Article
        fields = [
            "title",
            "slug",
            "body",
            "status",
            "published_at",
            "tags",
        ]
        widgets = {
            "body": forms.Textarea(attrs={"rows": 10}),
            "published_at": forms.DateTimeInput(
                attrs={"type": "datetime-local"},
            ),
        }
        help_texts = {
            "slug": "URL-friendly identifier (unique). Example: hello-django",
        }

    def clean_title(self) -> str:
        title = self.cleaned_data["title"].strip()
        if len(title) < 3:
            raise forms.ValidationError("Title must be at least 3 characters.")
        return title

    def clean_slug(self) -> str:
        slug = self.cleaned_data["slug"].strip()
        normalized = slugify(slug)

        if not normalized:
            raise forms.ValidationError("Slug is required.")

        return normalized

    def clean(self):
        cleaned = super().clean()
        status = cleaned.get("status")
        published_at = cleaned.get("published_at")

        if status == Article.Status.PUBLISHED and not published_at:
            cleaned["published_at"] = timezone.now()

        return cleaned
```

### 11.6.2 Explain the important design decisions

#### Why we define `tags` explicitly
Django would generate a default widget for the ManyToMany field, but we often want:
- a better widget (`CheckboxSelectMultiple`) for small tag lists
- control over ordering and help text

In big systems, you’d often use `autocomplete_fields` in admin and maybe a select
widget in public forms. For learning and clarity, checkboxes are perfect.

#### Why `clean_slug` uses `slugify`
Users often type:
- spaces
- uppercase
- punctuation

`slugify` normalizes `"Hello Django!"` → `"hello-django"`.

This is a professional pattern because it reduces slug errors and improves URL
consistency.

#### Why `clean()` auto-sets `published_at`
You have a DB constraint requiring published articles to have `published_at`.

There are two options:
1. Reject submission with an error (“published_at required”)
2. Automatically set it on publish (common editorial workflow)

We choose (2) for better UX:
- user selects “Published”
- system sets published time automatically if missing

If your product requires explicit scheduling, you’d choose (1) and require it.

> Note: This is form-level behavior; you may later enforce similar rules in model
> methods or service layer to protect non-form writes.

---

## 11.7 Form Lifecycle (GET vs POST): The Core Pattern You’ll Use Forever

### 11.7.1 Unbound form (GET)
On GET, show an empty form:

```python
form = ArticleForm()
return render(request, "articles/form.html", {"form": form})
```

### 11.7.2 Bound form (POST)
On POST, bind submitted data:

```python
form = ArticleForm(request.POST)
if form.is_valid():
    form.save()
    return redirect(...)
```

If invalid:
- `form.errors` contains errors
- re-render the same template with the bound form
- user sees errors and their input preserved

---

## 11.8 Create Article View (FBV) + PRG + Messages

We’ll implement a create page at:

- `/articles/new/` (GET shows form, POST saves)

### 11.8.1 Add messages UI first (site-wide)

Django messages are enabled by default when:
- `django.contrib.messages` is in `INSTALLED_APPS` (it is)
- `MessageMiddleware` is in `MIDDLEWARE` (it is)
- messages context processor is enabled (it is)

Add a partial to display messages.

Create `templates/partials/_messages.html`:

```django
{% if messages %}
  <ul class="messages">
    {% for message in messages %}
      <li class="message {{ message.tags }}">{{ message }}</li>
    {% endfor %}
  </ul>
{% endif %}
```

Include it in `templates/base.html` inside `<main>` before content:

```django
<main class="container">
  {% include "partials/_messages.html" %}
  {% block content %}{% endblock %}
</main>
```

Add minimal CSS (optional) to `static/css/site.css`:

```css
.messages {
  list-style: none;
  padding: 0;
}

.message {
  padding: 0.75rem 1rem;
  margin: 0.5rem 0;
  background: #eef6ff;
  border: 1px solid #cfe5ff;
}

.message.success {
  background: #eefbe7;
  border-color: #c7f3a3;
}
```

### 11.8.2 Implement the create view
Edit `articles/views.py` (add new view functions; keep list/detail):

```python
from django.contrib import messages
from django.shortcuts import redirect, render
from django.views.decorators.http import require_http_methods

from .forms import ArticleForm


@require_http_methods(["GET", "POST"])
def article_create(request):
    if request.method == "POST":
        form = ArticleForm(request.POST)
        if form.is_valid():
            article = form.save()
            messages.success(request, "Article created successfully.")
            return redirect("articles:detail_html", slug=article.slug)
    else:
        form = ArticleForm()

    return render(
        request,
        "articles/form.html",
        {
            "form": form,
            "mode": "create",
        },
    )
```

### 11.8.3 Explain the PRG pattern (Post/Redirect/Get)
Notice we do:

- POST (save) → redirect (302) → GET (detail page)

This prevents:
- duplicate form submissions if user refreshes the page
- “Confirm form resubmission” browser warning

It also:
- gives a clean URL after submission
- makes UX more consistent

This is a standard professional web pattern.

---

## 11.9 Edit Article View (FBV) Using `instance=...`

Now implement edit at:

- `/articles/<slug>/edit/`

### 11.9.1 Why edit uses `instance`
`ModelForm(instance=article)` populates initial values.
`ModelForm(request.POST, instance=article)` updates that existing row.

### 11.9.2 Implement edit view
Add to `articles/views.py`:

```python
from django.contrib import messages
from django.shortcuts import get_object_or_404, redirect, render
from django.views.decorators.http import require_http_methods

from .forms import ArticleForm
from .models import Article


@require_http_methods(["GET", "POST"])
def article_edit(request, slug: str):
    article = get_object_or_404(Article, slug=slug)

    if request.method == "POST":
        form = ArticleForm(request.POST, instance=article)
        if form.is_valid():
            article = form.save()
            messages.success(request, "Article updated successfully.")
            return redirect("articles:detail_html", slug=article.slug)
    else:
        form = ArticleForm(instance=article)

    return render(
        request,
        "articles/form.html",
        {
            "form": form,
            "mode": "edit",
            "article": article,
        },
    )
```

---

## 11.10 Wire URLs (Order Matters!)

Update `articles/urls.py` carefully. Put `new/` and `edit/` routes **before** the
detail route if you have a catch-all slug route.

Example `articles/urls.py`:

```python
from django.urls import path

from . import views

app_name = "articles"

urlpatterns = [
    path("html/", views.article_list_html, name="list_html"),
    path("new/", views.article_create, name="create"),
    path("html/<slug:slug>/edit/", views.article_edit, name="edit"),
    path("html/<slug:slug>/", views.article_detail_html, name="detail_html"),
]
```

### Why this ordering matters
If you had a pattern like:

```python
path("<slug:slug>/", ...)
```

it could capture `new` as a slug. You must keep special routes (`new/`, `edit/`)
more specific and earlier.

---

## 11.11 Create the Form Template (Industry-Standard Manual Rendering)

### 11.11.1 Why manual rendering is preferred for real apps
Django provides:
- `{{ form.as_p }}`
- `{{ form.as_table }}`
- `{{ form.as_ul }}`

These are great for quick prototypes, but they:
- limit markup control
- make design systems harder
- make consistent error UI harder

Professional projects typically render fields manually.

### 11.11.2 Create `articles/templates/articles/form.html`
Create `articles/templates/articles/form.html`:

```django
{% extends "base.html" %}

{% block title %}
  {% if mode == "edit" %}Edit Article{% else %}New Article{% endif %}
{% endblock %}

{% block content %}
  <h1>
    {% if mode == "edit" %}
      Edit: {{ article.title }}
    {% else %}
      New Article
    {% endif %}
  </h1>

  <form method="post">
    {% csrf_token %}

    {% if form.non_field_errors %}
      <div class="form-errors">
        {{ form.non_field_errors }}
      </div>
    {% endif %}

    <div class="field">
      <label for="{{ form.title.id_for_label }}">Title</label>
      {{ form.title }}
      {% if form.title.help_text %}<small>{{ form.title.help_text }}</small>{% endif %}
      {% for err in form.title.errors %}<div class="error">{{ err }}</div>{% endfor %}
    </div>

    <div class="field">
      <label for="{{ form.slug.id_for_label }}">Slug</label>
      {{ form.slug }}
      {% if form.slug.help_text %}<small>{{ form.slug.help_text }}</small>{% endif %}
      {% for err in form.slug.errors %}<div class="error">{{ err }}</div>{% endfor %}
    </div>

    <div class="field">
      <label for="{{ form.body.id_for_label }}">Body</label>
      {{ form.body }}
      {% for err in form.body.errors %}<div class="error">{{ err }}</div>{% endfor %}
    </div>

    <div class="field">
      <label for="{{ form.status.id_for_label }}">Status</label>
      {{ form.status }}
      {% for err in form.status.errors %}<div class="error">{{ err }}</div>{% endfor %}
    </div>

    <div class="field">
      <label for="{{ form.published_at.id_for_label }}">Published at</label>
      {{ form.published_at }}
      {% for err in form.published_at.errors %}
        <div class="error">{{ err }}</div>
      {% endfor %}
      <small>Leave empty when drafting. Auto-set when publishing (in this workbook).</small>
    </div>

    <div class="field">
      <div>Tags</div>
      {{ form.tags }}
      {% for err in form.tags.errors %}<div class="error">{{ err }}</div>{% endfor %}
    </div>

    <button type="submit">
      {% if mode == "edit" %}Save changes{% else %}Create{% endif %}
    </button>
  </form>
{% endblock %}
```

### 11.11.3 Explain key template elements

- `{% csrf_token %}`: mandatory for POST forms.
- `form.non_field_errors`: errors raised in `clean()` that don’t belong to one field.
- `form.<field>.errors`: per-field errors (list).
- `id_for_label`: ensures label points to the right input id for accessibility.

---

## 11.12 Update Navigation to Link to “New Article”

Edit `templates/partials/_nav.html` to add:

```django
<a href="{% url 'articles:create' %}">New Article</a>
```

This is another reason URL namespacing matters.

---

## 11.13 Understanding `is_valid()`, `cleaned_data`, and Error Structures

### 11.13.1 What `is_valid()` really does
`form.is_valid()` runs:

1. field validation:
   - type conversion (string → datetime, etc.)
   - required checks
   - max_length/min_length
2. `clean_<field>()` methods
3. `clean()` method (cross-field validation)
4. populates:
   - `form.cleaned_data` (normalized Python values)
   - `form.errors` (structured errors)

### 11.13.2 A debugging pattern (dev only)
Inside a view when a form is invalid, you can log:

```python
# dev-only debugging
print(form.errors)
```

But in real apps, you’d use logging, not print.

---

## 11.14 `commit=False` (Critical Pattern When You Must Set Fields)

You don’t need it yet, but this is *industry critical*.

Example: later you add `author = ForeignKey(User)` and want:

- author always equals current user
- never user-submitted

Pattern:

```python
article = form.save(commit=False)
article.author = request.user
article.save()
form.save_m2m()
```

Explanation:
- `commit=False` creates the model instance without saving to DB yet.
- You set fields programmatically.
- Save it.
- Then `save_m2m()` saves many-to-many relationships (needs PK).

Even if you’re not using this now, you must understand it early.

---

## 11.15 File Uploads (Correct Handling; Minimal + Realistic)

File uploads change **three** things:
1. HTML form must have `enctype="multipart/form-data"`
2. Django view must pass `request.FILES` into the form
3. You must configure media storage (`MEDIA_ROOT`, `MEDIA_URL`) if you store files

### 11.15.1 Minimal demo (without changing your models)
Add a demo form that accepts a file and just reports its name/size (not storing).

Create `pages/forms_upload.py` (or `pages/forms.py`):

```python
from django import forms


class UploadDemoForm(forms.Form):
    file = forms.FileField()

    def clean_file(self):
        f = self.cleaned_data["file"]
        max_bytes = 2 * 1024 * 1024
        if f.size > max_bytes:
            raise forms.ValidationError("File too large (max 2 MB).")
        return f
```

View (add to `pages/views.py`):

```python
from django.shortcuts import render
from django.views.decorators.http import require_http_methods

from .forms_upload import UploadDemoForm


@require_http_methods(["GET", "POST"])
def upload_demo(request):
    if request.method == "POST":
        form = UploadDemoForm(request.POST, request.FILES)
        if form.is_valid():
            f = form.cleaned_data["file"]
            return render(
                request,
                "pages/upload_done.html",
                {"filename": f.name, "size": f.size},
            )
    else:
        form = UploadDemoForm()

    return render(request, "pages/upload_demo.html", {"form": form})
```

Templates:

`pages/templates/pages/upload_demo.html`:

```django
{% extends "base.html" %}

{% block title %}Upload Demo{% endblock %}

{% block content %}
  <h1>Upload Demo</h1>

  <form method="post" enctype="multipart/form-data">
    {% csrf_token %}
    {{ form.file.label_tag }}
    {{ form.file }}
    {% for err in form.file.errors %}<div class="error">{{ err }}</div>{% endfor %}
    <button type="submit">Upload</button>
  </form>
{% endblock %}
```

`pages/templates/pages/upload_done.html`:

```django
{% extends "base.html" %}

{% block title %}Upload Done{% endblock %}

{% block content %}
  <h1>Upload Received</h1>
  <ul>
    <li>Filename: {{ filename }}</li>
    <li>Size: {{ size }} bytes</li>
  </ul>
{% endblock %}
```

URL (`pages/urls.py`):

```python
path("upload-demo/", views.upload_demo, name="upload_demo"),
```

#### Why this matters even without saving files
You learn the correct mechanics:
- `request.FILES`
- multipart encoding
- file validation (size limits)

Later you’ll store files using model fields and storage backends.

---

## 11.16 Tests for Form Views (Create, Edit, Validation Errors)

Create `articles/tests_forms.py`:

```python
from django.test import TestCase
from django.urls import reverse
from django.utils import timezone

from articles.models import Article, Tag


class ArticleFormViewTests(TestCase):
    def setUp(self):
        self.tag = Tag.objects.create(name="Django", slug="django")

    def test_create_get_renders_form(self):
        url = reverse("articles:create")
        response = self.client.get(url)

        self.assertEqual(response.status_code, 200)
        self.assertContains(response, "<form", html=False)

    def test_create_post_creates_article_and_redirects(self):
        url = reverse("articles:create")
        response = self.client.post(
            url,
            {
                "title": "New Article",
                "slug": "new-article",
                "body": "Hello world body content.",
                "status": Article.Status.PUBLISHED,
                "published_at": "",
                "tags": [self.tag.id],
            },
        )

        self.assertEqual(response.status_code, 302)
        self.assertEqual(Article.objects.count(), 1)

        a = Article.objects.get(slug="new-article")
        self.assertEqual(a.status, Article.Status.PUBLISHED)
        self.assertIsNotNone(a.published_at)
        self.assertEqual(list(a.tags.values_list("slug", flat=True)), ["django"])

    def test_create_post_shows_errors(self):
        url = reverse("articles:create")
        response = self.client.post(
            url,
            {
                "title": "",
                "slug": "",
                "body": "short",
                "status": Article.Status.DRAFT,
                "published_at": "",
            },
        )

        self.assertEqual(response.status_code, 200)
        self.assertEqual(Article.objects.count(), 0)
        self.assertContains(response, "Slug is required.")
```

### Notes on testing forms
- Successful POST should redirect (PRG). We assert 302.
- Invalid POST should return 200 and re-render form with errors (no redirect).
- We test a meaningful error message to ensure validation is wired.

---

## 11.17 Common Form Pitfalls (and Exact Fixes)

### Pitfall A: CSRF 403 errors on POST
Cause:
- missing `{% csrf_token %}` in the form template
Fix:
- add `{% csrf_token %}` inside `<form>` and ensure middleware is enabled.

### Pitfall B: Form “loses user input” when invalid
Cause:
- you created a new unbound form on POST instead of using posted data
Fix:
- on POST, use `form = ArticleForm(request.POST)` so it stays bound.

### Pitfall C: File upload shows empty file
Cause:
- missing `enctype="multipart/form-data"` or missing `request.FILES`
Fix:
- add enctype
- pass `request.FILES` to the form constructor

### Pitfall D: IntegrityError on save (unique or constraints)
Cause:
- form validation doesn’t cover model constraints, or concurrency race conditions
Fix:
- add form validation for domain rules (like published_at requirement)
- keep DB constraints anyway (they are your final safety net)
- optionally catch `IntegrityError` and add a friendly error message

---

## 11.18 Exercises (Do These Before Proceeding)

1. Add a “Delete article” flow (safe):
   - GET shows confirm page
   - POST deletes and redirects to list
   - must include CSRF token
   - return 405 for wrong methods

2. Improve the form UX:
   - add placeholder attributes to title and slug widgets
   - add CSS classes to inputs via widgets

3. Add form-level validation:
   - if status is draft, slug must start with `draft-` (toy rule)
   - display a non-field error when violated

4. Add a search form on the articles list page using GET:
   - fields: `q`, `tag`
   - when submitted, URL becomes `?q=...&tag=...`
   - explain why GET is correct here and POST is not

5. Write tests for edit:
   - create an Article in setup
   - GET edit page 200
   - POST changes title and redirects
   - verify DB updated

---

## 11.19 Chapter Summary

- HTML forms are HTTP request generators; POST changes state, GET filters/searches.
- Django Forms provide structured validation and error display.
- ModelForms are the standard way to create/edit models safely.
- Always use CSRF tokens for POST forms.
- Use PRG (POST → redirect → GET) to prevent duplicate submissions.
- Messages framework gives clean feedback UX.
- File uploads require multipart encoding and `request.FILES`.
- Tests should cover both success and invalid paths.

---

Next chapter: **Part II — Chapter 12: Authentication and Authorization**  
We’ll add login/logout, protect create/edit views, introduce permissions, and apply
object-level ownership rules the way real Django apps do.

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='10. admin_site.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='12. authentication_and_authorization.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
